In this fourth and final notebook, we will go over how to leverage the SDK to directly work interactively with a Ray cluster during development.

To Do: I tried adding the flan code in the interactive notebook but hit some errors. They need to be resolved to see if we can run the training in a distributed manner.  The bitsandbytes package doesn't work because of CUDA and Pytorch version.

In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [2]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "XX",
    server = "https://api.et-cluster.6mwp.p1.openshiftapps.com:6443",
    skip_tls=False
)
auth.login()

'Logged into "https://api.et-cluster.6mwp.p1.openshiftapps.com:6443" as "shanand@redhat.com" using the token provided.\n\nYou have access to 113 projects, the list has been suppressed. You can list all projects with \'oc projects\'\n\nUsing project "opendatahub".\n'

Once again, let's start by running through the same cluster setup as before:

In [3]:
# Create and configure our cluster object (and appwrapper)
cluster = Cluster(ClusterConfiguration(
    name='finetuneflan',
    namespace='default',
    min_worker=2,
    max_worker=2,
    min_cpus=1,
    max_cpus=2,
    min_memory=2,
    max_memory=8,
    gpu=1,
    instascale=False,
))

Written to: finetuneflan.yaml


In [4]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

Waiting for requested resources to be set up...


KeyboardInterrupt: 

In [5]:
cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   finetuneflan                                        Inactive ❌   │ 
 │                                                                     │ 
 │   URI: ray://finetuneflan-head-svc.default.svc:10001                │ 
 │                                                                     │ 
 │   ]8;id=510497;http://ray-dashboard-finetuneflan-default.apps.et-cluster.6mwp.p1.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 │                      Cluster Resources                              │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮          │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │          │ 
 │   │            │  │                                      │          │ 
 │   │  2    2    │  │  2~8         1           1           │          │ 
 │   │            │  │                                      │          │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯          │ 
 ╰─────────────────────────────────────────────────────────────────────╯

RayCluster(name='finetuneflan', status=<CodeFlareClusterStatus.STARTING: 2>, min_workers=2, max_workers=2, worker_mem_min=2, worker_mem_max=8, worker_cpu=1, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-finetuneflan-default.apps.et-cluster.6mwp.p1.openshiftapps.com')

This time we will demonstrate another potential method of use: working with the Ray cluster interactively.

Using the SDK, we can get both the Ray cluster URI and dashboard URI:

In [6]:
ray_dashboard_uri = cluster.cluster_dashboard_uri()
ray_cluster_uri = cluster.cluster_uri()
print(ray_dashboard_uri)
print(ray_cluster_uri)

http://ray-dashboard-finetuneflan-default.apps.et-cluster.6mwp.p1.openshiftapps.com
ray://finetuneflan-head-svc.default.svc:10001


Now we can connect directly to our Ray cluster via the Ray python client:

In [8]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray
from ray.air.config import ScalingConfig

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for model training
runtime_env = {"pip": ["transformers",
                       "datasets",
                       "evaluate",
                       "pyarrow<7.0.0",
                       "accelerate",
                       "bitsandbytes",
                       "loralib",
                       "py7zr",
                       "tensorboard",
                       "peft"], 
              "env_vars": {"HF_HOME":"huggingface"}}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env, _temp_dir="huggingface")

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


Now that we are connected (and have passed in some package requirements), let's try writing some training code for a DistilBERT transformer model via HuggingFace (using IMDB dataset):

In [18]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.train.huggingface import HuggingFaceTrainer
    
    from datasets import load_dataset, concatenate_datasets
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
    from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

    model_name = "ybelkada/flan-t5-xl-sharded-bf16"

    #model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    dataset = load_dataset("samsum")

    print(f"Train dataset size: {len(dataset['train'])}")
    print(f"Test dataset size: {len(dataset['test'])}")
    
    #### COMPUTE MAX SEQ LEN ##########
    # The maximum total input sequence length after tokenization.
    # Sequences longer than this will be truncated, sequences shorter will be padded.
    conc_dataset = concatenate_datasets([dataset["train"], dataset["test"]])

    
    tokenized_inputs = conc_dataset.map(lambda x: tokenizer(x["dialogue"],
                                                            truncation=True),
                                        batched=True,
                                        remove_columns=["dialogue", "summary"])
    
    input_lengths = [len(x) for x in tokenized_inputs["input_ids"]]
    # take 85 percentile of max length for better utilization
    max_source_length = int(np.percentile(input_lengths, 85))
    print(f"Max source length: {max_source_length}")

    # The maximum total sequence length for target text after tokenization.
    # Sequences longer than this will be truncated, sequences shorter will be padded."
    tokenized_targets = conc_dataset.map(lambda x: tokenizer(x["dialogue"],
                                                            truncation=True),
                                        batched=True,
                                        remove_columns=["dialogue", "summary"])  
    target_lengths = [len(x) for x in tokenized_targets["input_ids"]]
    # take 90 percentile of max length for better utilization
    max_target_length = int(np.percentile(target_lengths, 90))
    print(f"Max target length: {max_target_length}")
    
    #### PREPROCESS DATA ##########
    
    def preprocess_function(sample,padding="max_length"):
        # add prefix to the input for t5
        inputs = ["summarize: " + item for item in sample["dialogue"]]

        # tokenize inputs
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length":
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
    print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

    ray_train_ds = ray.data.from_huggingface(tokenized_dataset['train'])
    ray_evaluation_ds = ray.data.from_huggingface(tokenized_dataset['test'])

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model_name = "ybelkada/flan-t5-xl-sharded-bf16"
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["q", "v"],
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.SEQ_2_SEQ_LM
        )
        # prepare int-8 model for training
        model = prepare_model_for_int8_training(model)

        # add LoRA adaptor
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
        
        from transformers import DataCollatorForSeq2Seq

        # we want to ignore tokenizer pad token in the loss
        label_pad_token_id = -100
        # Data collator
        data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=8
        )
        
        output_dir="/tmp/flan/test"

        # Define training args
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            auto_find_batch_size=True,
            learning_rate=1e-3, # higher learning rate
            num_train_epochs=5,
            logging_dir=f"{output_dir}/logs",
            logging_strategy="steps",
            logging_steps=500,
            save_strategy="no",
            report_to="tensorboard",
        )

        trainer = Seq2SeqTrainer(model=model,
                                args=training_args,
                                data_collator=data_collator,
                                train_dataset=tokenized_dataset["train"])
        
        return trainer

    scaling_config = ScalingConfig(num_workers=2, use_gpu=True) #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()

Once we want to test our code out, we can run the training function we defined above remotely on our Ray cluster:

In [19]:
#call the above cell as a remote ray function
ray.get(train_fn.remote())

(train_fn pid=2288) 
(train_fn pid=2288) ===================================BUG REPORT===================================
(train_fn pid=2288) Welcome to bitsandbytes. For bug reports, please run
(train_fn pid=2288) 
(train_fn pid=2288) python -m bitsandbytes
(train_fn pid=2288) 
(train_fn pid=2288)  and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
(train_fn pid=2288) ================================================================================
(train_fn pid=2288) bin /tmp/ray/session_2023-05-24_13-35-49_647135_7/runtime_resources/pip/1ea979819413e33c98849c7e365e5e151f8a8356/virtualenv/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
(train_fn pid=2288) /tmp/ray/session_2023-05-24_13-35-49_647135_7/runtime_resources/pip/1ea979819413e33c98849c7e365e5e151f8a8356/virtualenv/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
(train_fn pid=2288) CUDA SETUP: 

(train_fn pid=2288) /tmp/ray/session_2023-05-24_13-35-49_647135_7/runtime_resources/pip/1ea979819413e33c98849c7e365e5e151f8a8356/virtualenv/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(train_fn pid=2288)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(train_fn pid=2288) huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
(train_fn pid=2288) To disable this warning, you can either:
(train_fn pid=2288) 	- Avoid using `tokenizers` before the fork if possible
(train_fn pid=2288) 	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(train_fn pid=2288) Found cached dataset samsum (/home/ray/workspace/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)
100%|██████████| 3/3 [00:00<00:00, 680.49it/s]
(train_fn pid=2288) Loading cached processed dataset at /home/ray/workspace/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-0d5be1d47aabc667.arrow


(train_fn pid=2288) Train dataset size: 14732
(train_fn pid=2288) Test dataset size: 819
(train_fn pid=2288) Max source length: 255


...

(TrainTrainable pid=613, ip=10.128.14.50) /tmp/ray/session_2023-05-24_13-35-49_647135_7/runtime_resources/pip/1ea979819413e33c98849c7e365e5e151f8a8356/virtualenv/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(TrainTrainable pid=613, ip=10.128.14.50)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(train_fn pid=2288) == Status ==
(train_fn pid=2288) Current time: 2023-05-24 14:34:26 (running for 00:00:05.39)
(train_fn pid=2288) Memory usage on this node: 6.0/30.9 GiB 
(train_fn pid=2288) Using FIFO scheduling algorithm.
(train_fn pid=2288) Resources requested: 1.0/6 CPUs, 2.0/2 GPUs, 0.0/22.35 GiB heap, 0.0/6.54 GiB objects
(train_fn pid=2288) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-05-24_14-34-20
(train_fn pid=2288) Number of trials: 1/1 (1 RUNNING)
(train_fn pid=2288) +--------------------------------+----------+-------+
(train_fn pid=2288) | Trial name                     | status   | loc   |
(train_fn pid=2288) |--------------------------------+----------+-------|
(train_fn pid=2288) | HuggingFaceTrainer_be877_00000 | RUNNING  |       |
(train_fn pid=2288) +--------------------------------+----------+-------+
(train_fn pid=2288) 
(train_fn pid=2288) 
(train_fn pid=2288) Result for HuggingFaceTrainer_be877_00000:
(train_fn pid=2288)   trial_id: be877_0000

(TrainTrainable pid=613, ip=10.128.14.50) 2023-05-24 14:34:26,170	ERROR serialization.py:371 -- [Errno 2] Failed to open local file '/home/ray/workspace/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-b332bfb65957e3fe.arrow'. Detail: [errno 2] No such file or directory
(TrainTrainable pid=613, ip=10.128.14.50) Traceback (most recent call last):
(TrainTrainable pid=613, ip=10.128.14.50)   File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/_private/serialization.py", line 369, in deserialize_objects
(TrainTrainable pid=613, ip=10.128.14.50)     obj = self._deserialize_object(data, metadata, object_ref)
(TrainTrainable pid=613, ip=10.128.14.50)   File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/_private/serialization.py", line 252, in _deserialize_object
(TrainTrainable pid=613, ip=10.128.14.50)     return self._deserialize_msgpack_data(data, metadata_fields)
(TrainTrainable pid=613, ip=10.128.14.50)   File "/

RayTaskError(TrainingFailedError): [36mray::train_fn()[39m (pid=2288, ip=10.128.10.87)
ray.tune.error.TuneError: Failure # 1 (occurred at 2023-05-24_14-34-26)
[36mray::train_fn()[39m (pid=2288, ip=10.128.10.87)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, [36mray::_Inner.__init__()[39m (pid=613, ip=10.128.14.50, repr=HuggingFaceTrainer)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/trainable/util.py", line 339, in setup
    setup_kwargs[k] = parameter_registry.get(prefix + k)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/registry.py", line 234, in get
    return ray.

(train_fn pid=2288) 2023-05-24 14:34:26,284	ERROR tune.py:773 -- Trials did not complete: [HuggingFaceTrainer_be877_00000]
(train_fn pid=2288) 2023-05-24 14:34:26,284	INFO tune.py:777 -- Total run time: 5.50 seconds (5.39 seconds for the tuning loop).


Once complete, we can bring our Ray cluster down and clean up:

In [1]:
cluster.down()

NameError: name 'cluster' is not defined

In [ ]:
auth.logout()